# Oddstradamus
### Good odds and where to find them

### Data Processing

As already mentioned in the introduction, one of the biggest parts of this project is the processing of the historical game data into useful data that can be used for prediction. In order to realise this, a large number of features and auxiliary features were created, then tailored to the EDA and the model building and stored accordingly for further use. The corresponding procedure is described in this notebook.

In [1]:
#import packages
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

First, the merged data set is read in and those columns are filtered which are to be classified as usable.

In [2]:
# loading the merged dataframe
df = pd.read_csv('Data/Results2013_2021.csv')

In [3]:
# selecting the necessary columns of the original data set
df = df[['Div', 'Season', 'HomeTeam','AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR','B365H','B365D','B365A', 'BWH','BWD','BWA', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA']]
df

,Div,Season,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,...,B365A,BWH,BWD,BWA,IWH,IWD,IWA,WHH,WHD,WHA
0,B1,13/14,Club Brugge,Charleroi,2.0,0.0,H,NaN,NaN,NaN,...,9.00,1.35,4.60,7.00,1.30,4.40,8.5,1.30,5.0,9.00
1,B1,13/14,Bergen,Cercle Brugge,1.0,1.0,D,NaN,NaN,NaN,...,4.20,1.90,3.60,3.40,1.90,3.30,3.5,1.91,3.3,3.80
2,B1,13/14,Genk,Oostende,3.0,0.0,H,NaN,NaN,NaN,...,6.00,1.40,4.40,6.25,1.40,4.00,6.5,1.50,4.2,5.50
3,B1,13/14,Kortrijk,Oud-Heverlee Leuven,1.0,0.0,H,NaN,NaN,NaN,...,3.80,1.80,3.40,4.00,1.90,3.20,3.6,1.91,3.3,3.80
4,B1,13/14,Lierse,Waregem,1.0,2.0,A,NaN,NaN,NaN,...,2.30,3.00,3.40,2.10,3.00,3.10,2.2,3.00,3.1,2.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59410,T1,20/21,Ankaragucu,Goztep,3.0,0.0,H,17.0,8.0,9.0,...,2.25,2.65,3.50,2.45,2.85,3.45,2.4,2.80,3.3,2.45
59411,T1,20/21,Erzurum BB,Trabzonspor,0.0,0.0,D,11.0,8.0,3.0,...,2.00,3.50,3.25,2.10,3.70,3.20,2.1,3.75,3.2,2.05
59412,T1,20/21,Gaziantep,Denizlispor,2.0,0.0,H,14.0,6.0,3.0,...,4.00,1.85,3.50,4.10,1.85,3.55,4.2,1.85,3.4,4.20
59413,T1,20/21,Fenerbahce,Genclerbirligi,1.0,2.0,A,16.0,7.0,6.0,...,11.00,1.25,5.75,10.50,1.25,6.25,12.0,1.22,5.8,12.00


As can already be seen in the simple output of the dataset, the raw dataset contains some 'NaN' entries. To see how many missing values are involved, we output df.info(). At the same time we get an overview of the respective data types.

In [4]:
# Output the number of missing values and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59415 entries, 0 to 59414
Data columns (total 31 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Div       59315 non-null  object 
 1   Season    59415 non-null  object 
 2   HomeTeam  59315 non-null  object 
 3   AwayTeam  59315 non-null  object 
 4   FTHG      59313 non-null  float64
 5   FTAG      59313 non-null  float64
 6   FTR       59313 non-null  object 
 7   HS        42619 non-null  float64
 8   AS        42619 non-null  float64
 9   HST       42619 non-null  float64
 10  AST       42619 non-null  float64
 11  HF        41279 non-null  float64
 12  AF        41279 non-null  float64
 13  HC        42619 non-null  float64
 14  AC        42619 non-null  float64
 15  HY        45028 non-null  float64
 16  AY        45029 non-null  float64
 17  HR        45029 non-null  float64
 18  AR        45028 non-null  float64
 19  B365H     59173 non-null  float64
 20  B365D     59173 non-null  fl

It can be seen that a large part of the columns are incomplete. Nevertheless, we keep this circumstance for the time being and do not clean the data at this point, as this is needed to generate data for the prediction model from the historical match data. The cleaning will be done at a later date.

In [5]:
# shape of the data set before feature engineering
df.shape

(59415, 31)

### Feature Engineering 1.0

In order to generate additional match data, the number of goals, shots, goal kicks and corners (hereinafter referred to simply as events) per match is determined. Furthermore, the average odd is calculated on the basis of the available odds of the bookmakers for simplification purposes. In addition, the maximum odds of all bookmakers are recorded. This will become relevant at a later date for the formation of the long-term strategy.

In [6]:
# total number of the specific event in the game
df['total_goals'] = df['FTHG'] + df['FTAG']
df['total_shots'] = df['HS'] + df['AS']
df['total_sot'] = df['HST'] + df['AST']
df['total_corners'] = df['HC'] + df['AC']

In [7]:
# average odds per game
df['AvgHome'] = (df['B365H'] + df['BWH'] + df['IWH'] + df['WHH']) / 4
df['AvgDraw'] = (df['B365D'] + df['BWD'] + df['IWD'] + df['WHD']) / 4
df['AvgAway'] = (df['B365A'] + df['BWA'] + df['IWA'] + df['WHA']) / 4

In [8]:
# highest odds per game
df['MaxHome'] = df[['B365H','BWH','IWH','WHH']].max(axis=1)
df['MaxDraw'] = df[['B365D','BWD','IWD','WHD']].max(axis=1)
df['MaxAway'] = df[['B365A','BWA','IWA','WHA']].max(axis=1)

First of all, the number of events per season is cumulated for each team. In this way, useful information from past matches can be collected for each team. Since the number of events of the corresponding match is subtracted for each match, only data that is known at the start of the match is available.

In [9]:
# cumulative number of goals
df['GoalsHome'] = df.groupby(["HomeTeam","Season"])['FTHG'].cumsum()
df['GoalsAway'] = df.groupby(["AwayTeam","Season"])['FTAG'].cumsum()
# known number of goals before the start of the match 
df['GoalsHome_before'] = df['GoalsHome'] - df['FTHG']
df['GoalsAway_before'] = df['GoalsAway'] - df['FTAG']
# cumulative number of goals conceded
df['Home_Goals_Conceded'] = df.groupby(['HomeTeam',"Season"])['FTAG'].cumsum()
df['Away_Goals_Conceded'] = df.groupby(['AwayTeam',"Season"])['FTHG'].cumsum()
# known number of goals conceded before the start of a match
df['Home_Goals_Conceded_before'] = df['Home_Goals_Conceded'] - df['FTAG']
df['Away_Goals_Conceded_before'] = df['Away_Goals_Conceded'] - df['FTHG']

# cumulative total number of goals in the match
df['TotalGoals_Home'] = df.groupby(['HomeTeam',"Season"])['total_goals'].cumsum()
df['TotalGoals_Away'] = df.groupby(['AwayTeam',"Season"])['total_goals'].cumsum()
# known total number of goals in the match before the start of a match
df['TotalGoals_Home_before'] = df['TotalGoals_Home'] - df['total_goals']
df['TotalGoals_Away_before'] = df['TotalGoals_Away'] - df['total_goals']


# cumulative number of shots
df['ShotsHome'] = df.groupby(["HomeTeam","Season"])['HS'].cumsum()
df['ShotsAway'] = df.groupby(["AwayTeam","Season"])['AS'].cumsum()
# known number of shots before the start of the match 
df['ShotsHome_before'] = df['ShotsHome'] - df['HS']
df['ShotsAway_before'] = df['ShotsAway'] - df['AS']
# cumulative number of shots conceded
df['Home_Shots_Conceded'] = df.groupby(['HomeTeam',"Season"])['AS'].cumsum()
df['Away_Shots_Conceded'] = df.groupby(['AwayTeam',"Season"])['HS'].cumsum()
# known number of shots conceded before the start of a match
df['Home_Shots_Conceded_before'] = df['Home_Shots_Conceded'] - df['AS']
df['Away_Shots_Conceded_before'] = df['Away_Shots_Conceded'] - df['HS']

# cumulative total number of shots in the match
df['TotalShots_Home'] = df.groupby(['HomeTeam',"Season"])['total_shots'].cumsum()
df['TotalShots_Away'] = df.groupby(['AwayTeam',"Season"])['total_shots'].cumsum()
# known total number of shots in the match before the start of a match
df['TotalShots_Home_before'] = df['TotalShots_Home'] - df['total_shots']
df['TotalShots_Away_before'] = df['TotalShots_Away'] - df['total_shots']


# cumulative number of shots on target
df['SOTHome'] = df.groupby(["HomeTeam","Season"])['HST'].cumsum()
df['SOTAway'] = df.groupby(["AwayTeam","Season"])['AST'].cumsum()
# known number of shots on target before the start of the match 
df['SOTHome_before'] = df['SOTHome'] - df['HST']
df['SOTAway_before'] = df['SOTAway'] - df['AST']
# cumulative number of shots on target conceded
df['Home_SOT_Conceded'] = df.groupby(['HomeTeam',"Season"])['AST'].cumsum()
df['Away_SOT_Conceded'] = df.groupby(['AwayTeam',"Season"])['HST'].cumsum()
# known number of shots on target conceded before the start of a match
df['Home_SOT_Conceded_before'] = df['Home_SOT_Conceded'] - df['AST']
df['Away_SOT_Conceded_before'] = df['Away_SOT_Conceded'] - df['HST']

# cumulative total number of shots on target in the match
df['TotalSOT_Home'] = df.groupby(['HomeTeam',"Season"])['total_sot'].cumsum()
df['TotalSOT_Away'] = df.groupby(['AwayTeam',"Season"])['total_sot'].cumsum()
# known total number of shots on target in the match before the start of a match
df['TotalSOT_Home_before'] = df['TotalSOT_Home'] - df['total_sot']
df['TotalSOT_Away_before'] = df['TotalSOT_Away'] - df['total_sot']


# cumulative number of corners
df['CornersHome'] = df.groupby(["HomeTeam","Season"])['HC'].cumsum()
df['CornersAway'] = df.groupby(["AwayTeam","Season"])['AC'].cumsum()
# known number of corners before the start of the match 
df['CornersHome_before'] = df['CornersHome'] - df['HC']
df['CornersAway_before'] = df['CornersAway'] - df['AC']
# cumulative number of corners conceded
df['Home_Corners_Conceded'] = df.groupby(['HomeTeam',"Season"])['AC'].cumsum()
df['Away_Corners_Conceded'] = df.groupby(['AwayTeam',"Season"])['HC'].cumsum()
# known number of corners conceded before the start of a match
df['Home_Corners_Conceded_before'] = df['Home_Corners_Conceded'] - df['AC']
df['Away_Corners_Conceded_before'] = df['Away_Corners_Conceded'] - df['HC']

# cumulative total number of corners in the match
df['TotalCorners_Home'] = df.groupby(['HomeTeam',"Season"])['total_corners'].cumsum()
df['TotalCorners_Away'] = df.groupby(['AwayTeam',"Season"])['total_corners'].cumsum()
# known total number of corners in the match before the start of a match
df['TotalCorners_Home_before'] = df['TotalCorners_Home'] - df['total_corners']
df['TotalCorners_Away_before'] = df['TotalCorners_Away'] - df['total_corners']

In order to be able to form average values per match, the total number of matches as well as the number of matches per season are first determined and then the targeted average values are calculated.

In [10]:
# total number of games per team / per season
df['game_count_home'] = df.groupby(['HomeTeam'])['FTHG'].cumcount()
df['game_count_away'] = df.groupby(['AwayTeam'])['FTAG'].cumcount()
df['game_count_home_season'] = df.groupby(['HomeTeam',"Season"])['FTHG'].cumcount()
df['game_count_away_season'] = df.groupby(['AwayTeam',"Season"])['FTAG'].cumcount()

In [11]:
# average occurrence of the specific event per team / per season
df['goals_avg_home_season'] = df['GoalsHome_before'] / df['game_count_home_season']
df['goals_avg_away_season'] = df['GoalsAway_before'] / df['game_count_away_season']
df['goals_conceded_avg_home_season'] = df['Home_Goals_Conceded_before'] / df['game_count_home_season']
df['goals_conceded_avg_away_season'] = df['Away_Goals_Conceded_before'] / df['game_count_away_season']
df['goals_total_avg_home_season'] = df['TotalGoals_Home_before'] / df['game_count_home_season']
df['goals_total_avg_away_season'] = df['TotalGoals_Away_before'] / df['game_count_away_season']


df['shots_avg_home_season'] = df['ShotsHome_before'] / df['game_count_home_season']
df['shots_avg_away_season'] = df['ShotsAway_before'] / df['game_count_away_season']
df['shots_conceded_avg_home_season'] = df['Home_Shots_Conceded_before'] / df['game_count_home_season']
df['shots_conceded_avg_away_season'] = df['Away_Shots_Conceded_before'] / df['game_count_away_season']
df['shots_total_avg_home_season'] = df['TotalShots_Home_before'] / df['game_count_home_season']
df['shots_total_avg_away_season'] = df['TotalShots_Away_before'] / df['game_count_away_season']


df['sot_avg_home_season'] = df['SOTHome_before'] / df['game_count_home_season']
df['sot_avg_away_season'] = df['SOTAway_before'] / df['game_count_away_season']
df['sot_conceded_avg_home_season'] = df['Home_SOT_Conceded_before'] / df['game_count_home_season']
df['sot_conceded_avg_away_season'] = df['Away_SOT_Conceded_before'] / df['game_count_away_season']
df['sot_total_avg_home_season'] = df['TotalSOT_Home_before'] / df['game_count_home_season']
df['sot_total_avg_away_season'] = df['TotalSOT_Away_before'] / df['game_count_away_season']


df['corners_avg_home_season'] = df['CornersHome_before'] / df['game_count_home_season']
df['corners_avg_away_season'] = df['CornersAway_before'] / df['game_count_away_season']
df['corners_conceded_avg_home_season'] = df['Home_Corners_Conceded_before'] / df['game_count_home_season']
df['corners_conceded_avg_away_season'] = df['Away_Corners_Conceded_before'] / df['game_count_away_season']
df['corners_total_avg_home_season'] = df['TotalCorners_Home_before'] / df['game_count_home_season']
df['corners_total_avg_away_season'] = df['TotalCorners_Away_before'] / df['game_count_away_season']

In order to be able to determine the number of wins, draws and loses per season, auxiliary features are first created and used to determine the cumulative numbers of events. Furthermore, these features can also be used to determine the cumulative points.

In [12]:
# auxiliary column to determine the cumulative win / draw / lose number
df['WinHome'] = df['FTR'].apply(lambda x: 1 if x == 'H' else 0)
df['DrawHome'] = df['FTR'].apply(lambda x: 1 if x == 'D' else 0)
df['LoseHome'] = df['FTR'].apply(lambda x: 1 if x == 'A' else 0)

df['WinAway'] = df['FTR'].apply(lambda x: 1 if x == 'A' else 0)
df['DrawAway'] = df['FTR'].apply(lambda x: 1 if x == 'D' else 0)
df['LoseAway'] = df['FTR'].apply(lambda x: 1 if x == 'H' else 0)

# cumulative occurrence of the specific event / per season
df['Total_wins_home'] = df.groupby(['HomeTeam'])['WinHome'].cumsum()
df['Total_draws_home'] = df.groupby(['HomeTeam'])['DrawHome'].cumsum()
df['Total_loses_home'] = df.groupby(['HomeTeam'])['LoseHome'].cumsum()
df['Total_wins_home_season'] = df.groupby(['HomeTeam', 'Season'])['WinHome'].cumsum()
df['home_wins_season'] = df.groupby(['HomeTeam', 'Season'])['Total_wins_home_season'].shift()
df['Total_draws_home_season'] = df.groupby(['HomeTeam', 'Season'])['DrawHome'].cumsum()
df['home_draws_season'] = df.groupby(['HomeTeam', 'Season'])['Total_draws_home_season'].shift()
df['Total_loses_home_season'] = df.groupby(['HomeTeam', 'Season'])['LoseHome'].cumsum()
df['home_loses_season'] = df.groupby(['HomeTeam', 'Season'])['Total_loses_home_season'].shift()

df['Total_wins_away'] = df.groupby(['AwayTeam'])['WinAway'].cumsum()
df['Total_draws_away'] = df.groupby(['AwayTeam'])['DrawAway'].cumsum()
df['Total_loses_away'] = df.groupby(['AwayTeam'])['LoseAway'].cumsum()
df['Total_wins_away_season'] = df.groupby(['AwayTeam', 'Season'])['WinAway'].cumsum()
df['away_wins_season'] = df.groupby(['AwayTeam', 'Season'])['Total_wins_away_season'].shift()
df['Total_draws_away_season'] = df.groupby(['AwayTeam', 'Season'])['DrawAway'].cumsum()
df['away_draws_season'] = df.groupby(['AwayTeam', 'Season'])['Total_draws_away_season'].shift()
df['Total_loses_away_season'] = df.groupby(['AwayTeam', 'Season'])['LoseAway'].cumsum()
df['away_loses_season'] = df.groupby(['AwayTeam', 'Season'])['Total_loses_away_season'].shift()

# determine the number of points / per season
df['home_points'] = (3 * df['home_wins_season']) + df['home_draws_season']
df['away_points'] = (3 * df['away_wins_season']) + df['away_draws_season']

This is followed by the calculation of the cumulative average odds per team per season.

In [13]:
# determine the average odd of the specific event per team / per season
df['homewin_cum_odd'] = df.groupby(['HomeTeam'])['AvgHome'].cumsum()
df['homewin_avg_odd'] = df['homewin_cum_odd'] / (df['game_count_home'] + 1)
df['homewin_cum_odd_season'] = df.groupby(['HomeTeam','Season'])['AvgHome'].cumsum()
df['homewin_avg_odd_season'] = df['homewin_cum_odd_season'] / (df['game_count_home_season'] + 1)

df['homedraw_cum_odd'] = df.groupby(['HomeTeam'])['AvgDraw'].cumsum()
df['homedraw_avg_odd'] = df['homedraw_cum_odd'] / (df['game_count_home'] + 1)
df['homedraw_cum_odd_season'] = df.groupby(['HomeTeam', 'Season'])['AvgDraw'].cumsum()
df['homedraw_avg_odd_season'] = df['homedraw_cum_odd_season'] / (df['game_count_home_season'] + 1)

df['homelose_cum_odd'] = df.groupby(['HomeTeam'])['AvgAway'].cumsum()
df['homelose_avg_odd'] = df['homelose_cum_odd'] / (df['game_count_home'] + 1)
df['homelose_cum_odd_season'] = df.groupby(['HomeTeam', 'Season'])['AvgAway'].cumsum()
df['homelose_avg_odd_season'] = df['homelose_cum_odd_season'] / (df['game_count_home_season'] + 1)

df['awaywin_cum_odd'] = df.groupby(['AwayTeam'])['AvgAway'].cumsum()
df['awaywin_avg_odd'] = df['awaywin_cum_odd'] / (df['game_count_away'] + 1)
df['awaywin_cum_odd_season'] = df.groupby(['AwayTeam', 'Season'])['AvgAway'].cumsum()
df['awaywin_avg_odd_season'] = df['awaywin_cum_odd_season'] / (df['game_count_away_season'] + 1)

df['awaydraw_cum_odd'] = df.groupby(['AwayTeam'])['AvgDraw'].cumsum()
df['awaydraw_avg_odd'] = df['awaydraw_cum_odd'] / (df['game_count_away'] + 1)
df['awaydraw_cum_odd_season'] = df.groupby(['AwayTeam', 'Season'])['AvgDraw'].cumsum()
df['awaydraw_avg_odd_season'] = df['awaydraw_cum_odd_season'] / (df['game_count_away_season'] + 1)

df['awaylose_cum_odd'] = df.groupby(['AwayTeam'])['AvgHome'].cumsum()
df['awaylose_avg_odd'] = df['awaylose_cum_odd'] / (df['game_count_away'] + 1)
df['awaylose_cum_odd_season'] = df.groupby(['AwayTeam', 'Season'])['AvgHome'].cumsum()
df['awaylose_avg_odd_season'] = df['awaylose_cum_odd_season'] / (df['game_count_away_season'] + 1)

In order to be able to determine the average odds for events that have occurred, auxiliary features are first created on the basis of which the desired values can be calculated. In this context, it is important to note the danger of unintentional data leakage, which can be avoided by postponing the information to the next match of the corresponding team.

In [14]:
# determine the odds of the events occurring
df['arrived_homewin'] = df['WinHome'] * df['AvgHome']
df['arrived_homedraw'] = df['DrawHome'] * df['AvgDraw']
df['arrived_homelose'] = df['LoseHome'] * df['AvgAway']

df['arrived_awaywin'] = df['WinAway'] * df['AvgAway']
df['arrived_awaydraw'] = df['DrawAway'] * df['AvgDraw']
df['arrived_awaylose'] = df['LoseAway'] * df['AvgHome']

In [15]:
# determine the average odd of occurrence per team / per season
# Generate the cumulative average odd per team per season if event occurs
df['cum_arr_homewin_odd'] = df.groupby(['HomeTeam'])['arrived_homewin'].cumsum()
df['avg_arr_homewin_odd'] = df['cum_arr_homewin_odd'] / df['Total_wins_home']
df['home_prev_win_all'] = df.groupby(['HomeTeam'])['avg_arr_homewin_odd'].shift()
df['cum_arr_homewin_odd_season'] = df.groupby(['HomeTeam','Season'])['arrived_homewin'].cumsum()
df['avg_arr_homewin_odd_season'] = df['cum_arr_homewin_odd_season'] / df['Total_wins_home_season']
df['home_prev_win_season'] = df.groupby(['HomeTeam', 'Season'])['avg_arr_homewin_odd_season'].shift()

df['cum_arr_homedraw_odd'] = df.groupby(['HomeTeam'])['arrived_homedraw'].cumsum()
df['avg_arr_homedraw_odd'] = df['cum_arr_homedraw_odd'] / df['Total_draws_home']
df['home_prev_draw_all'] = df.groupby(['HomeTeam'])['avg_arr_homedraw_odd'].shift()
df['cum_arr_homedraw_odd_season'] = df.groupby(['HomeTeam','Season'])['arrived_homedraw'].cumsum()
df['avg_arr_homedraw_odd_season'] = df['cum_arr_homedraw_odd_season'] / df['Total_draws_home_season']
df['home_prev_draw_season'] = df.groupby(['HomeTeam', 'Season'])['avg_arr_homedraw_odd_season'].shift()

df['cum_arr_homelose_odd'] = df.groupby(['HomeTeam'])['arrived_homelose'].cumsum()
df['avg_arr_homelose_odd'] = df['cum_arr_homelose_odd'] / df['Total_loses_home']
df['home_prev_lose_all'] = df.groupby(['HomeTeam'])['avg_arr_homelose_odd'].shift()
df['cum_arr_homelose_odd_season'] = df.groupby(['HomeTeam', 'Season'])['arrived_homelose'].cumsum()
df['avg_arr_homelose_odd_season'] = df['cum_arr_homelose_odd_season'] / df['Total_loses_home_season']
df['home_prev_lose_season'] = df.groupby(['HomeTeam', 'Season'])['avg_arr_homelose_odd_season'].shift()


df['cum_arr_awaywin_odd'] = df.groupby(['AwayTeam'])['arrived_awaywin'].cumsum()
df['avg_arr_awaywin_odd'] = df['cum_arr_awaywin_odd'] / df['Total_wins_away']
df['away_prev_win_all'] = df.groupby(['AwayTeam'])['avg_arr_awaywin_odd'].shift()
df['cum_arr_awaywin_odd_season'] = df.groupby(['AwayTeam', 'Season'])['arrived_awaywin'].cumsum()
df['avg_arr_awaywin_odd_season'] = df['cum_arr_awaywin_odd_season'] / df['Total_wins_away_season']
df['away_prev_win_season'] = df.groupby(['AwayTeam', 'Season'])['avg_arr_awaywin_odd_season'].shift()

df['cum_arr_awaydraw_odd'] = df.groupby(['AwayTeam'])['arrived_awaydraw'].cumsum()
df['avg_arr_awaydraw_odd'] = df['cum_arr_awaydraw_odd'] / df['Total_draws_away']
df['away_prev_draw_all'] = df.groupby(['AwayTeam'])['avg_arr_awaydraw_odd'].shift()
df['cum_arr_awaydraw_odd_season'] = df.groupby(['AwayTeam','Season'])['arrived_awaydraw'].cumsum()
df['avg_arr_awaydraw_odd_season'] = df['cum_arr_awaydraw_odd_season'] / df['Total_draws_away_season']
df['away_prev_draw_season'] = df.groupby(['AwayTeam', 'Season'])['avg_arr_awaydraw_odd_season'].shift()

df['cum_arr_awaylose_odd'] = df.groupby(['AwayTeam'])['arrived_awaylose'].cumsum()
df['avg_arr_awaylose_odd'] = df['cum_arr_awaylose_odd'] / df['Total_loses_away']
df['away_prev_lose_all'] = df.groupby(['AwayTeam'])['avg_arr_awaylose_odd'].shift()
df['cum_arr_awaylose_odd_season'] = df.groupby(['AwayTeam','Season'])['arrived_awaylose'].cumsum()
df['avg_arr_awaylose_odd_season'] = df['cum_arr_awaylose_odd_season'] / df['Total_loses_away_season']
df['away_prev_lose_season'] = df.groupby(['AwayTeam', 'Season'])['avg_arr_awaylose_odd_season'].shift()

In addition, so-called match ratings are formed. Rating systems are quantitative measures of the superiority of one team over the other. In this context, the difference between the two opponents is determined with regard to various events.

In [16]:
# determine the team-specific difference between achieving and allowing the events
df['team_diff_goals_home_before'] = df['GoalsHome_before'] - df['Home_Goals_Conceded_before']
df['team_diff_shots_home_before'] = df['ShotsHome_before'] - df['Home_Shots_Conceded_before']
df['team_diff_SOT_home_before'] = df['SOTHome_before'] - df['Home_SOT_Conceded_before']
df['team_diff_corners_home_before'] = df['CornersHome_before'] - df['Home_Corners_Conceded_before']

df['team_diff_goals_away_before'] = df['GoalsAway_before'] - df['Away_Goals_Conceded_before']
df['team_diff_shots_away_before'] = df['ShotsAway_before'] - df['Away_Shots_Conceded_before']
df['team_diff_SOT_away_before'] = df['SOTAway_before'] - df['Away_SOT_Conceded_before']
df['team_diff_corners_away_before'] = df['CornersAway_before'] - df['Away_Corners_Conceded_before']

In [17]:
# determine the difference between the events achieved between home and away team
df['cross_diff_goals_before'] = df['GoalsHome_before'] - df['GoalsAway_before']
df['cross_diff_shots_before'] = df['ShotsHome_before'] - df['ShotsAway_before']
df['cross_diff_sot_before'] = df['SOTHome_before'] - df['SOTAway_before']
df['cross_diff_corners_before'] = df['CornersHome_before'] - df['CornersAway_before']

In [18]:
# determine the difference between the events achieved between home and away team
df['cross_goal_home'] = df['GoalsHome_before'] - df['Away_Goals_Conceded_before']
df['cross_shots_home'] = df['ShotsHome_before'] - df['Away_Shots_Conceded_before']
df['cross_SOT_home'] = df['SOTHome_before'] - df['Away_SOT_Conceded_before']
df['cross_corners_home'] = df['CornersHome_before'] - df['Away_Corners_Conceded_before']

df['cross_goal_away'] = df['GoalsAway_before'] - df['Home_Goals_Conceded_before']
df['cross_shots_away'] = df['ShotsAway_before'] - df['Home_Shots_Conceded_before']
df['cross_SOT_away'] = df['SOTAway_before'] - df['Home_SOT_Conceded_before']
df['cross_corners_away'] = df['CornersAway_before'] - df['Home_Corners_Conceded_before']

In [19]:
# determine the difference between the events allowed between home and away team
df['cross_diff_goals_Conceded_before'] = df['Home_Goals_Conceded_before'] - df['Away_Goals_Conceded_before']
df['cross_diff_shots_Conceded_before'] = df['Home_Shots_Conceded_before'] - df['Away_Shots_Conceded_before']
df['cross_diff_sot_Conceded_before'] = df['Home_SOT_Conceded_before'] - df['Away_SOT_Conceded_before']
df['cross_diff_corners_Conceded_before'] = df['Home_Corners_Conceded_before'] - df['Away_Corners_Conceded_before']

Since a sports bet can either be won or lost, it only has two different outcomes instead of three, unlike a football match. Accordingly, three-way outcomes are converted into two-way outcomes.

In [20]:
# Change three-way to two-way results
df['two_way_h_odd'] = df['WinHome'].apply(lambda x: '1' if x == 1 else 'X2')
df['two_way_a_odd'] = df['WinAway'].apply(lambda x: '2' if x == 1 else '1X')
df['two_way_home'] = df['WinHome'].apply(lambda x: 'Win' if x == 1 else 'No win')
df['two_way_away'] = df['WinAway'].apply(lambda x: 'Win' if x == 1 else 'No win')

A betting odds contains two pieces of information. Firstly, the probability that the event will occur: the lower it is, the more likely it is that the event will actually occur. And secondly, the odds can be used to calculate the profit after multiplication by the stake. A detailed description of the relationship follows in the notebook EDA.

We determine the real probabilities per game. Provided the bookmakers would deliver fair odds, a summation of the probabilities of the three-way outcome would add up to 100%. To build a dataframe, we need any matches whose outcome is considered by the bookmakers to have a real probability of over 50%. In addition, the summed probabilities can be used to calculate the odds for the 'double chance' outcome. Double chance odds pull two of the three events together to form a two-way bet. For example, a bet can be won if you bet on double chance home win or draw (1X) and the match actually ends with one of the two outcomes.

In [21]:
# calculate the actual probabilities based on the betting odds
df['proba_total'] = (1 / df['AvgHome']) + (1 / df['AvgDraw']) + (1 / df['AvgAway'])
df['real_proba_home'] = (1 / df['AvgHome']) / df['proba_total']
df['real_proba_away'] = (1 / df['AvgAway']) / df['proba_total']
df['dc_away'] = 1 / (df['proba_total'] - (1 / df['AvgHome']))
df['dc_home'] = 1 / (df['proba_total'] - (1 / df['AvgAway']))

### Data Cleaning

A lot of information is not available for games that are mainly a little further in the past. Therefore, some lines had to be dropped completely, while for others the missing values are filled with 0 without making the data unusable. Games that take place at the beginning of the season do not contain season data to avoid data leakage, so these missing values can be filled with 0 without hesitation.

In [22]:
# dropping the low information lines
df = df[df['AvgHome'].notna()]
df = df[df['AvgAway'].notna()]
df = df[df['GoalsHome_before'].notna()]
df = df[df['ShotsHome_before'].notna()]

In [23]:
# Fill in the empty values with 0
columns = ['goals_avg_home_season', 'goals_avg_away_season', 'goals_conceded_avg_home_season', 'goals_conceded_avg_away_season',
           'goals_total_avg_home_season', 'goals_total_avg_away_season', 'shots_avg_home_season', 'shots_avg_away_season',
           'shots_conceded_avg_home_season', 'shots_conceded_avg_away_season', 'shots_total_avg_home_season',
           'shots_total_avg_away_season', 'sot_avg_home_season', 'sot_avg_away_season', 'sot_conceded_avg_home_season',
           'sot_conceded_avg_away_season', 'sot_total_avg_home_season', 'sot_total_avg_away_season', 'corners_avg_home_season',
           'corners_avg_away_season', 'corners_conceded_avg_home_season', 'corners_conceded_avg_away_season',
           'corners_total_avg_home_season', 'corners_total_avg_away_season', 'home_wins_season', 'home_draws_season', 
           'home_loses_season', 'away_wins_season', 'away_draws_season', 'away_loses_season', 'home_points', 'away_points',
           'home_prev_win_all', 'home_prev_win_season', 'home_prev_draw_all', 'home_prev_draw_season', 'home_prev_lose_all',
           'home_prev_lose_season', 'away_prev_win_all', 'away_prev_win_season', 'away_prev_draw_all', 'away_prev_draw_season', 
           'away_prev_lose_all', 'away_prev_lose_season']

for column in columns:
    df[column] = df[column].fillna(0)

### Feature Engineering 2.0

Since some features depended on all fields being filled first so that they could take on the correct values, the following features were only created afterwards.

In [24]:
# Calculate the difference between the odd and the cumulative odd of the team when the event occurs.
df['diff_exp_val_home'] = df['AvgHome'] - df['home_prev_win_season']
df['diff_exp_val_away'] = df['AvgAway'] - df['away_prev_win_season']

In [25]:
# Calculate the points difference between hometeam and awayteam
df['cross_diff_points_before'] = df['home_points'] - df['away_points']

In [26]:
# shape after feature engineering (including auxiliary columns)
df.shape

(42258, 245)

### Preparing the Dataframes

Since data leakage does not play a role in EDA, different data frames were created for EDA and Model Building. Only after the EDA data frame has been created are the problematic features removed and prepared for the model building.

#### EDA

In [27]:
# saving the modified dataframe for EDA
df.to_csv("Data/EDA.csv")

#### Modelling

In [28]:
# create dummie-variables
division_dummies = pd.get_dummies(df['Div'], prefix='Div', drop_first=True)
season_dummies = pd.get_dummies(df['Season'], prefix='Season', drop_first=True)

In [29]:
# appending the dummie-variables
df = pd.concat((df, division_dummies, season_dummies), axis=1)

In [30]:
# dropping unnecessary columns for the model building
df = df.drop(['Div', 'Season', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 
             'HY', 'AY', 'HR', 'AR', 'total_goals', 'total_shots', 'total_sot', 'total_corners', 'GoalsHome', 'GoalsAway', 
             'Home_Goals_Conceded', 'Away_Goals_Conceded', 'TotalGoals_Home', 'TotalGoals_Away', 'ShotsHome',
             'ShotsAway', 'Home_Shots_Conceded', 'Away_Shots_Conceded', 'TotalShots_Home', 'TotalShots_Away',
             'SOTHome', 'SOTAway', 'Home_SOT_Conceded', 'Away_SOT_Conceded', 'TotalSOT_Home', 'TotalSOT_Away',
             'CornersHome', 'CornersAway', 'Home_Corners_Conceded', 'Away_Corners_Conceded', 'TotalCorners_Home',
             'TotalCorners_Away', 'game_count_home', 'game_count_away', 'game_count_home_season', 'game_count_away_season',
             'arrived_homewin', 'arrived_homewin', 'arrived_homedraw', 'arrived_homelose', 'arrived_awaywin',
             'arrived_awaydraw', 'arrived_awaylose', 'two_way_home', 'two_way_away', 'avg_arr_homewin_odd',
             'avg_arr_homewin_odd_season', 'avg_arr_homedraw_odd', 'avg_arr_homedraw_odd_season', 'avg_arr_homelose_odd',
             'avg_arr_homelose_odd_season', 'avg_arr_awaywin_odd', 'avg_arr_awaywin_odd_season', 'avg_arr_awaydraw_odd',
             'avg_arr_awaydraw_odd_season', 'avg_arr_awaylose_odd', 'avg_arr_awaylose_odd_season', 'proba_total',
             'Total_wins_home', 'Total_wins_away', 'Total_draws_home', 'Total_draws_away', 'Total_loses_home', 
             'Total_loses_away', 'Total_wins_home_season', 'Total_draws_home_season', 'Total_loses_home_season',
             'Total_wins_away_season', 'Total_draws_away_season', 'Total_loses_away_season', 'B365H', 'B365D', 
             'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'WinHome',
             'DrawHome', 'LoseHome', 'WinAway', 'DrawAway', 'LoseAway', 'homewin_cum_odd', 'homewin_cum_odd_season', 
             'homedraw_cum_odd', 'homedraw_cum_odd_season', 'homelose_cum_odd', 'homelose_cum_odd_season', 'awaywin_cum_odd',
             'awaywin_cum_odd_season', 'awaydraw_cum_odd', 'awaydraw_cum_odd_season', 'awaylose_cum_odd',
             'awaylose_cum_odd_season', 'cum_arr_homewin_odd', 'cum_arr_homewin_odd_season', 'cum_arr_homedraw_odd',
             'cum_arr_homedraw_odd_season', 'cum_arr_homelose_odd', 'cum_arr_homelose_odd_season', 'cum_arr_awaywin_odd',
             'cum_arr_awaywin_odd_season', 'cum_arr_awaydraw_odd', 'cum_arr_awaydraw_odd_season', 'cum_arr_awaylose_odd',
             'cum_arr_awaylose_odd_season'], axis=1)

In [31]:
# shape after preparing 
df.shape

(42258, 149)

To predict the football matches, we examine different datasets. Since we want to determine the outcome in two-way mode and predict either a home win or an away win or the respective double chance counter event, home and away matches have to be trained in different models. In addition, different compositions of the data set will be used to investigate to what extent these have an influence on the prediction or the improvement of the long-term strategy. Accordingly, the following 6 data frames are generated:
- Data set containing only matches with a real probability of a home win of more than 50%.
- Data set containing only matches with a real probability of an away win above 50%.
- Data set which only contains teams whose average home win probability is below a certain odd level
- Data set which only contains teams whose average away win probability is below a certain odd level
- Full data set (home win prediction)
- Full data set (away win prediction)

##### 50%-Dataframe

In [32]:
# reduce the data frame to the object of investigation / unnecessary columns
df_h = df[(df['real_proba_home'] >= 0.5)].reset_index()
df_a = df[(df['real_proba_away'] >= 0.5)].reset_index()

df_h = df_h.drop(['HomeTeam', 'AwayTeam', 'index', 'real_proba_home', 'real_proba_away', 'two_way_a_odd', 'dc_home'], axis=1)
df_a = df_a.drop(['HomeTeam', 'AwayTeam', 'index', 'real_proba_home', 'real_proba_away', 'two_way_h_odd', 'dc_away'], axis=1)

In [33]:
# saving the modified dataframe for the model building
df_h.to_csv('Data/model_home.csv')
df_a.to_csv('Data/model_away.csv')

##### Favorite-Dataframe

In [34]:
# reduce the data frame to the object of investigation / drop unnecessary columns
home_mean = (df.groupby(['HomeTeam'])['AvgHome'].mean())
away_mean = (df.groupby(['AwayTeam'])['AvgAway'].mean())

mean_h = home_mean.to_frame()
exp_val_h = mean_h[(mean_h['AvgHome'] >= 1.0) & (mean_h['AvgHome'] <= 1.55)]
picked_teams_h = list(exp_val_h.index.values)

mean_a = away_mean.to_frame()
exp_val_a = mean_a[(mean_a['AvgAway'] >= 1.0) & (mean_a['AvgAway'] <= 1.65)]
picked_teams_a = list(exp_val_a.index.values)

df_home_fav = df[df['HomeTeam'].isin(picked_teams_h)]
df_away_fav = df[df['AwayTeam'].isin(picked_teams_a)]

df_home_fav = df_home_fav.drop(['HomeTeam', 'AwayTeam', 'real_proba_home', 'real_proba_away', 'two_way_a_odd', 'dc_home'], axis = 1)
df_away_fav = df_away_fav.drop(['HomeTeam', 'AwayTeam', 'real_proba_home', 'real_proba_away', 'two_way_h_odd', 'dc_away'], axis = 1)

In [35]:
# saving the modified dataframe for the model building
df_home_fav.to_csv('Data/model_home_fav.csv')
df_away_fav.to_csv('Data/model_away_fav.csv')

##### Full-Dataframe

In [36]:
# drop unnecessary columns
df = df.drop(['HomeTeam', 'AwayTeam', 'real_proba_home', 'real_proba_away'], axis=1)

In [37]:
# saving the modified dataframe for the model building
df.to_csv('Data/model_full.csv')

In Notebook Model Preprocessing, the columns that will be used in the model are described before the split between training and test data is performed.